## Step 2 - Database Engineering
1. Read Cleaned measurements and station CSV files
2. Use the engine and connection string to create a database called hawaii.sqlite
3. Use declarative_base and create ORM classes for each table and define primary keys:
    * Measurements table
    * Stations table
4. Create the tables and the database using creat_all

In [2]:
import pandas as pd
import sqlalchemy
from datetime import datetime
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, ForeignKey, Integer, String, Numeric, Text, Float, Date
from sqlalchemy.orm import relationship

#### 1. Read Hawaii Measurements and Station CSV's into Dataframes

In [3]:
measures_df=pd.DataFrame(pd.read_csv("Resources/clean_hawaii_measurements.csv"))
station_df=pd.DataFrame(pd.read_csv("Resources/clean_hawaii_stations.csv"))

In [4]:
measures_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.00,73


In [5]:
measures_df.dtypes

station     object
date        object
prcp       float64
tobs         int64
dtype: object

In [6]:
measures_df["date"]=[datetime.strptime(d,"%Y-%m-%d") for d in measures_df["date"]]
measures_df.dtypes

station            object
date       datetime64[ns]
prcp              float64
tobs                int64
dtype: object

In [7]:
station_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


#### 2. Use the engine and connection string to create a database called hawaii.sqlite. 

In [8]:
engine=create_engine("sqlite:///hawaii.sqlite",echo=False)

In [9]:
conn=engine.connect()

#### 3. Use declarative_base and create ORM classes for each table and define primary keys:
* Measurements table
* Stations table

In [10]:
Base=declarative_base()

class Station(Base):
    __tablename__="stations"
    
    id=Column(Integer,primary_key=True)
    station=Column(Text)
    name=Column(Text)
    latitude=Column(Numeric)
    longitude=Column(Numeric)
    elevation=Column(Numeric)
    measures=relationship("Measurements",back_populates="stat")  
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
class Measurement(Base):
    __tablename__="measurements"
    
    id=Column(Integer,primary_key=True)
    station=Column(Text,ForeignKey("stations.station"))
    date=Column(Date)                            
    prcp=Column(Numeric)
    tobs=Column(Integer)
    stat=relationship("Station",back_populates="measures")
    
    def __repr__(self):
        return f"id={self.id},name={self.name}"
    

Questions:
* When creating multiple tables, how to distinguish which foreign keys go with which tables
* What is the purpose of the __repr__
* Should datetime format be used for the date?
* How to check for duplicate records?
* What is the "reflect" about?

#### 4. Create the tables and the database using creat_all

In [11]:
#Create tables in Database
Base.metadata.create_all(engine)   
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()


In [12]:
# Save reference to each table as a variable
station_table=sqlalchemy.Table('stations',metadata,autoload=True)
measurement_table=sqlalchemy.Table('measurements',metadata,autoload=True)

In [13]:
# Remove pre-existing data for purposes of running code multiple times
# This would not be used in a production environment
conn.execute(station_table.delete())
conn.execute(measurement_table.delete())

In [14]:
station_data=station_df.to_dict(orient="records")
measurement_data=measures_df.to_dict(orient="records")


In [15]:
# Populate the table in SQL
conn.execute(station_table.insert(),  station_data)
conn.execute(measurement_table.insert(), measurement_data)

In [16]:
conn.execute("SELECT * FROM stations limit 5").fetchall()


[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [17]:
conn.execute("SELECT * FROM measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0, 63),
 (3, 'USC00519397', '2010-01-03', 0, 74),
 (4, 'USC00519397', '2010-01-04', 0, 76),
 (5, 'USC00519397', '2010-01-06', 0, 73)]